Practice with Tensorflow from [here](http://www.ritchieng.com/machine-learning/deep-learning/tensorflow/deep-neural-nets/)

# GD #

In [13]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [14]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [15]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [16]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [17]:
# Gradient descent would take a while with full dataset
train_subset = 10000

#Create graph object by instantiation
graph = tf.Graph()

with graph.as_default():
    
    '''SET RANDOM SEED'''
    # Want results to be repeatable
    tf.set_random_seed(1)
    
    """INPUT DATA"""
    # Load train, valid and test data into constants
    # All datasets are constants as this is non-stochastic 
    # gradient descent
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    '''VARIABLES'''
    # Weights are initialized with random values
    # Biases are initalized with zeros
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    '''TRAINING COMPUTATION'''
    # X * W + b
    # Softmax >> Cross Entropy
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # Average of cross entropy across all training examples is loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    '''OPTIMIZER'''
    # Find the minimum of the loss using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    '''PREDICTIONS'''
    # Using train, test, and val data, report accuracy
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

Run computation and iterate

In [18]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Run computations
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step%100 == 0):
            print('Loss at step {}: {}'.format(step, l))
            print('Training accuracy: {:.1f}'.format(accuracy(predictions, train_labels[:train_subset, :])))
            
            # Calling eval() on valid_prediction is similar to run() except 
            # one numpy array is returned
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
            print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))
            

Initialized
Loss at step 0: 20.8992271423
Training accuracy: 7.1
Validation accuracy: 8.9
Test accuracy: 8.7
Loss at step 100: 2.45050215721
Training accuracy: 71.2
Validation accuracy: 70.3
Test accuracy: 77.8
Loss at step 200: 1.9473515749
Training accuracy: 74.3
Validation accuracy: 72.6
Test accuracy: 80.6
Loss at step 300: 1.67538130283
Training accuracy: 75.7
Validation accuracy: 73.8
Test accuracy: 81.7
Loss at step 400: 1.49323868752
Training accuracy: 76.7
Validation accuracy: 74.3
Test accuracy: 82.0
Loss at step 500: 1.36117422581
Training accuracy: 77.4
Validation accuracy: 74.8
Test accuracy: 82.4
Loss at step 600: 1.25924921036
Training accuracy: 78.0
Validation accuracy: 75.0
Test accuracy: 82.5
Loss at step 700: 1.17734396458
Training accuracy: 78.5
Validation accuracy: 75.4
Test accuracy: 82.8
Loss at step 800: 1.1097625494
Training accuracy: 78.9
Validation accuracy: 75.6
Test accuracy: 83.0


 # SGD#

Instead of `tf.constant`, will use `tf.placeholder` since the input will change with each batch.

In [19]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    tf.set_random_seed(1)
    
    """INPUT DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """VARIABLES"""
    # Weights initially random
    # Biases initialy zero
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    """TRAINING COMPUTATION"""
    # X * W + b
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    """PREDICTIONS"""
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick offset w/i training data which has been randomly chosen
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate mini-batch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dict telling session where to feed minibatch
        # Key is placeholder node of graph to be fed
        # Value is numpy array to feed it
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
            print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 20.7893486023
Minibatch accuracy: 6.2
Validation accuracy: 9.3
Test accuracy: 9.2
Minibatch loss at step 500: 1.77074170113
Minibatch accuracy: 75.8
Validation accuracy: 75.1
Test accuracy: 83.1
Minibatch loss at step 1000: 1.43556928635
Minibatch accuracy: 78.1
Validation accuracy: 76.2
Test accuracy: 84.2
Minibatch loss at step 1500: 0.845371305943
Minibatch accuracy: 79.7
Validation accuracy: 77.0
Test accuracy: 84.8
Minibatch loss at step 2000: 1.33343911171
Minibatch accuracy: 75.0
Validation accuracy: 77.5
Test accuracy: 85.2
Minibatch loss at step 2500: 0.984911859035
Minibatch accuracy: 78.1
Validation accuracy: 78.1
Test accuracy: 86.2
Minibatch loss at step 3000: 1.01682293415
Minibatch accuracy: 79.7
Validation accuracy: 78.1
Test accuracy: 86.2


# NN with 128 hidden nodes and relu activation

Train a neural net with 1 hidden layer containing 1024 nodes with relus.

In [22]:
num_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(1)
    
    """DATA"""
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """VARIABLES"""
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    """COMPUTE TRAINING"""
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
    
    """OPTIMIZER"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    """PREDICTIONS"""
    # Train prediction
    train_prediction = tf.nn.softmax(logits_2)
    
    # Valid prediction
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Test prediction
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits_2)

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Choose random offset in data
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare minibatch dict
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step%500 == 0):
            print('Minibatch loss at step {}: {}'.format(step, l))
            print('Minibatch accuracy: {:.1f}'.format(accuracy(predictions, batch_labels)))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), valid_labels)))
            print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 319.850372314
Minibatch accuracy: 6.2
Validation accuracy: 32.4
Test accuracy: 34.3
Minibatch loss at step 500: 14.0138378143
Minibatch accuracy: 80.5
Validation accuracy: 80.8
Test accuracy: 87.5
Minibatch loss at step 1000: 9.30784130096
Minibatch accuracy: 82.8
Validation accuracy: 81.2
Test accuracy: 87.4
Minibatch loss at step 1500: 5.70523262024
Minibatch accuracy: 82.0
Validation accuracy: 81.8
Test accuracy: 88.2
Minibatch loss at step 2000: 2.97010278702
Minibatch accuracy: 84.4
Validation accuracy: 82.6
Test accuracy: 89.0
Minibatch loss at step 2500: 2.14710092545
Minibatch accuracy: 83.6
Validation accuracy: 81.6
Test accuracy: 88.8
Minibatch loss at step 3000: 4.11144733429
Minibatch accuracy: 80.5
Validation accuracy: 82.7
Test accuracy: 89.1
